In [4]:
import numpy as np
import os
from components.network import Helper, Network

positives = Helper.get_positives(number_of_samples=500)
filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
np.savez(filepath, positives=positives)

C:\Users\grodo\Miniconda3\envs\Musicc\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\Users\grodo\Miniconda3\envs\Musicc\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Length of the specimen should be 660984


In [1]:
import os
import numpy as np
import librosa
from components.network import Helper

filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
file = np.load(filepath, allow_pickle=True)
positives = file['positives']
negatives = Helper.get_negatives(500, sample_length=660719)
train, test = Helper.prepare_dataset(positives, negatives, train_to_test_ratio=0.8)

train_mel = Helper.convert_to_feature(train, librosa.feature.melspectrogram)
test_mel = Helper.convert_to_feature(test, librosa.feature.melspectrogram)
x_train_mel, y_train_mel = Helper.get_x_y(train_mel)
x_test_mel, y_test_mel = Helper.get_x_y(test_mel)

train_tempo = Helper.convert_to_feature(train, librosa.feature.tempogram)
test_tempo = Helper.convert_to_feature(test, librosa.feature.tempogram)
x_train_tempo, y_train_tempo = Helper.get_x_y(train_tempo)
x_test_tempo, y_test_tempo = Helper.get_x_y(test_tempo)

train_rms = Helper.convert_to_feature(train, librosa.feature.rms)
test_rms = Helper.convert_to_feature(test, librosa.feature.rms)
x_train_rms, y_train_rms = Helper.get_x_y(train_rms)
x_test_rms, y_test_rms = Helper.get_x_y(test_rms)

train_mfcc = Helper.convert_to_feature(train, librosa.feature.mfcc)
test_mfcc = Helper.convert_to_feature(test, librosa.feature.mfcc)
x_train_mfcc, y_train_mfcc = Helper.get_x_y(train_mfcc)
x_test_mfcc, y_test_mfcc = Helper.get_x_y(test_mfcc)

train_chroma = Helper.convert_to_feature(train, librosa.feature.chroma_stft)
test_chroma = Helper.convert_to_feature(test, librosa.feature.chroma_stft)
x_train_chroma, y_train_chroma = Helper.get_x_y(train_chroma)
x_test_chroma, y_test_chroma = Helper.get_x_y(test_chroma)


Using plaidml.keras.backend backend.
C:\Users\grodo\Miniconda3\envs\Musicc\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


Number of all negative samples: 6965
Length of the specimen should be 660719


In [2]:
from components.network import Network

network_mel = Network(input_shape=x_train_mel[0].shape)
network_tempo = Network(input_shape=x_train_tempo[0].shape)
network_rms = Network(input_shape=x_train_rms[0].shape)
network_mfcc = Network(input_shape=x_train_mfcc[0].shape)
network_chroma = Network(input_shape=x_train_chroma[0].shape)

INFO:plaidml:Opening device "opencl_amd_gfx1010.0"


In [1]:
import os
from components.network import Network

network_mel = Network(input_shape=(128, 1291))
network_tempo = Network(input_shape=(384, 1291))
network_rms = Network(input_shape=(1, 1291))
network_mfcc = Network(input_shape=(20, 1291))
network_chroma = Network(input_shape=(12, 1291))

network_mel.load_model(os.path.join(os.path.pardir, 'models/model_mel'))
network_tempo.load_model(os.path.join(os.path.pardir, 'models/model_tempo'))
network_rms.load_model(os.path.join(os.path.pardir, 'models/model_rms'))
network_mfcc.load_model(os.path.join(os.path.pardir, 'models/model_mfcc'))
network_chroma.load_model(os.path.join(os.path.pardir, 'models/model_chroma'))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "opencl_amd_gfx1010.0"


In [5]:
network_mel.train(x_train_mel, y_train_mel, epochs=25, batch_size=100)
network_tempo.train(x_train_tempo, y_train_tempo, epochs=25, batch_size=100)
network_rms.train(x_train_rms, y_train_rms, epochs=25, batch_size=100)
network_mfcc.train(x_train_mfcc, y_train_mfcc, epochs=25, batch_size=100)
network_chroma.train(x_train_chroma, y_train_chroma, epochs=25, batch_size=100)

network_mel.evaluate(x_test_mel, y_test_mel)
network_tempo.evaluate(x_test_tempo, y_test_tempo)
network_rms.evaluate(x_test_rms, y_test_rms)
network_mfcc.evaluate(x_test_mfcc, y_test_mfcc)
network_chroma.evaluate(x_test_chroma, y_test_chroma)

Epoch 1/25
800/800 [==============================] - 3s 4ms/step - loss: 0.0027 - acc: 0.9987
Epoch 2/25
800/800 [==============================] - 1s 686us/step - loss: 0.0024 - acc: 0.9987

(0.024988484866917134, 0.99)

In [6]:
import os

network_mel.save_model(os.path.join(os.path.pardir, f'models/model_mel'))
network_tempo.save_model(os.path.join(os.path.pardir, f'models/model_tempo'))
network_rms.save_model(os.path.join(os.path.pardir, f'models/model_rms'))
network_mfcc.save_model(os.path.join(os.path.pardir, f'models/model_mfcc'))
network_chroma.save_model(os.path.join(os.path.pardir, f'models/model_chroma'))

In [ ]:
import librosa

from components.specimen import Evolution

#specimen_length = 660984
specimen_length = 660719
evolution = Evolution(neural_networks=[network_mel, network_tempo, network_rms, network_mfcc, network_chroma],
                      features_list=[librosa.feature.melspectrogram, librosa.feature.tempogram, librosa.feature.rms,
                                     librosa.feature.mfcc, librosa.feature.chroma_stft],
                      number_of_specimen=100,
                      specimen_length=specimen_length, target=None, mutation_chance=0.1, crossover_chance=0.1)
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)

Started fitness calculation --------------------
Started specimen conversion                    |
Finished specimen conversion                   v


In [ ]:
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)

